In [18]:
import os
import pandas as pd
import openpyxl

In [ ]:
qa_df = pd.read_csv("C:\Chat-App\deepseek_csv_20250428_de0f6c.csv", delimiter="@")
qa_df

,STT,Câu hỏi,Câu trả lời
0,1,Trong trường hợp lắp đặt điện thoại cửa ở khu ...,Không đấu dây khi có giông bão. Sử dụng ống bả...
1,2,Tại sao tài liệu yêu cầu sử dụng dây cáp một c...,Dây có vỏ bọc giúp chống nhiễu tín hiệu và đảm...
2,3,Mô tả chi tiết quy trình lắp đặt hộp rơ le để ...,1. Tháo dây từ điều khiển chính. 2. Nối dây kh...
3,4,Nếu điện thoại cửa bị đọng hơi nước làm mờ hìn...,Chờ nhiệt độ tăng để hơi nước tự bay hơi. Khôn...
4,5,Giải thích nguyên lý hoạt động của chế độ Zoom...,"Chế độ Zoom phóng đại hình ảnh bằng phần mềm, ..."
...,...,...,...
63,93,Cách lắp giá treo điều khiển chính?,Gắn lên tường phẳng và thẳng đứng
64,95,Cách thay đổi cài đặt [Wide/Zoom settings]?,Sử dụng điều khiển chính để chọn phạm vi nhìn
65,97,Cách đấu dây vào cổng S1/S2?,"Dùng tiếp điểm đóng khô N/O, 12V AC/DC"
66,99,Cách siết vít đầu nối AC?,"Lực siết 0,5 N·m {5,1 kgf·cm}"


In [27]:
from langchain_core.prompts import PromptTemplate
from langchain.schema import BaseRetriever, Document
from typing import List, Dict, Any
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains.llm import LLMChain
from rag_bot.vector_database.qdrant import QdrantVectorStore
from rag_bot.retrieval.retrieval import HybridSearch
from rag_bot.model.embedding import Embedding
from rag_bot.model.reranker import ReRanker
from rag_bot.utils.llm import GeminiLLM
from dotenv import load_dotenv
import os
from rag_bot.utils.prompt import SYSTEM_PROMPT
from rag_bot.rag import CustomQAChain, RAGPipeline

In [ ]:
### System Prompt

SYSTEM_PROMPT = """"
Bạn là **ProductReportBot**, một trợ lý ảo chuyên về **hỏi–đáp tài liệu phân tích báo cáo sản phẩm**. Nhiệm vụ của bạn:

1. **Chỉ** trả lời các câu hỏi liên quan đến nội dung của tài liệu phân tích báo cáo sản phẩm đã được cung cấp.  
2. **Không** trả lời hoặc hỗ trợ bất kỳ câu hỏi nào ngoài phạm vi đó (ví dụ: hỏi về thời tiết, cuộc sống cá nhân, kiến thức chung…).  
3. Nếu được hỏi điều gì không liên quan, hãy trả lời lịch sự:  
   > “Xin lỗi, tôi chỉ được phép trả lời các câu hỏi về tài liệu phân tích báo cáo sản phẩm.”  
4. Duy trì một **bộ đếm hình phạt** nội bộ cho các câu trả lời sai:
   - Bắt đầu từ 0.  
   - Nếu người dùng cho rằng câu trả lời của bạn không đúng hoặc không được hỗ trợ bởi tài liệu, tăng bộ đếm lên 1.  
   - Sau mỗi lần tăng, bạn phải:
     1. Thừa nhận sai sót:  
        > “Tôi hiểu, tôi sẽ sửa lại.”  
     2. Chạy lại quy trình ReAct để đưa ra câu trả lời đúng.  
   - Nếu bộ đếm hình phạt đạt **3**, ngay lập tức trả lời:  
     > “Tôi xin lỗi về các lỗi lặp lại. Vui lòng tham khảo chuyên gia để xem xét báo cáo này.”  
     và từ chối trả lời thêm.

---

Chat History:
{chat_history}

Context:
{context}

Question:
{question}

---

Bạn vận hành theo **mẫu ReAct** (Reasoning + Action): Tất cả phải có reply theo dạng text

> **USER:** {question}  
> **BOT (THINK):** “Tiếp theo tôi cần làm gì để hiểu và trả lời câu hỏi này?”  
> **BOT (ACTION):**  
> - Chọn một trong các hành động sau:
>   1. `SEARCH(query)` — tìm kiếm trong chỉ mục/tóm tắt báo cáo.  
>   2. `READ(section_id)` — đọc chi tiết phần đã chỉ định.  
>   3. `SUMMARIZE()` — tóm tắt lại thông tin đã thu thập.  
>   4. `ANSWER()` — đưa ra câu trả lời cuối cùng cho người dùng.  
> - Ví dụ:
>   ```
>   ACTION: SEARCH("doanh số tháng 3 cho Sản phẩm X")
>   ```
> **BOT (OBSERVE):** “Kết quả tìm kiếm cho thấy…”  
> **BOT (THINK):** “Từ thông tin này, tôi có thể trả lời rằng…”  
> **BOT (ACTION):**  
> ```
> ACTION: ANSWER()
> ```  
> **BOT (REPLY):** “Theo báo cáo, doanh số tháng 3 đạt 1.200 đơn vị, tăng 15% so với tháng trước.”

**Quy ước đặt tên hành động:**
- `SEARCH("từ khóa")`
- `READ("Mục 2.1")`
- `SUMMARIZE()`
- `ANSWER()`

**Lưu ý:**
- Mọi kết quả cuối cùng (sau `ANSWER()`) phải rõ ràng, ngắn gọn và trích dẫn phần mục liên quan khi cần.  
- Với các câu hỏi ngoài phạm vi, ngay lập tức dừng luồng ReAct và trả lời theo quy định lịch sự ở trên. 
- Phải tiến hành đến bước **BOT (REPLY):** 
- Theo dõi các câu trả lời sai qua bộ đếm hình phạt và tuân thủ nghiêm ngặt quy tắc xử lý phạt.  
"""

In [ ]:
GEMINI_API_KEY = "your_api_key_here"
QDRANT_API_KEY="your_api_key_here"
QDRANT_CLIENT_URL="https://0bcd2862-e8a5-4f2b-b1ac-4d0ed2af07ed.us-west-1-0.aws.cloud.qdrant.io:6333"

In [22]:
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
os.environ["QDRANT_API_KEY"] = QDRANT_API_KEY
os.environ["QDRANT_CLIENT_URL"] = QDRANT_CLIENT_URL

In [23]:
embedding_model = Embedding().get_embeddings()
sparse_model = Embedding().get_sparse_embeddings()
qdrant_store = QdrantVectorStore("my_collection", embedding_model, sparse_model)

c:\Users\Asus\anaconda3\envs\agent\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
class CustomQAChain:
    """Custom QA chain using HybridSearch for retrieval"""
    
    def __init__(self, hybrid_search: HybridSearch, llm, memory):
        self.hybrid_search = hybrid_search
        self.llm = llm
        self.memory = memory

        self.system_prompt = PromptTemplate.from_template(
            SYSTEM_PROMPT
        )
        
        self.llm_chain = LLMChain(llm=llm, prompt=self.system_prompt, verbose=True)

    def __call__(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        # Retrieve relevant documents
        docs = self.hybrid_search.get_relevant_documents(inputs["question"])
        
        # Get chat history from memory
        chat_history = self.memory.load_memory_variables({})["chat_history"]
        print("====== Chat History ======")
        
        # Format context from documents
        context = "\n".join([doc.page_content for doc in docs])
        
        # Generate answer
        response = self.llm_chain.run(
            question=inputs["question"],
            context=context,
            chat_history=chat_history
        )
        
        # Save to memory
        self.memory.save_context(
            {"question": inputs["question"]},
            {"answer": response}
        )
        
        return {"answer": response, "sources": [doc.metadata for doc in docs]}

class RAGPipeline:
    """Modified RAG Pipeline with HybridSearch integration"""
    
    def __init__(self, hybrid_search: HybridSearch, llm):
        self.hybrid_search = hybrid_search
        self.llm = llm
        self.sessions = {}
        self.create_session("1")

    def create_session(self, session_id: str):
        """Create a new session for the RAG pipeline"""
        if session_id in self.sessions:
            raise ValueError(f"Session {session_id} already exists.")
        
        memory = ConversationSummaryBufferMemory(
            llm=self.llm,
            max_token_limit=4096,
            memory_key="chat_history",
            return_messages=True
        )

        self.qa_chain = CustomQAChain(
            hybrid_search=self.hybrid_search,
            llm=self.llm,
            memory=memory
        )

        self.sessions[session_id] = {
            "memory": memory,
            "qa_chain": self.qa_chain
        }

    def reset_session(self, session_id: str) -> None:
        """
        Reset an existing session by clearing its memory.
        """
        if session_id not in self.sessions:
            raise ValueError(f"Session with ID '{session_id}' does not exist.")
        
        # Clear the memory for the session
        self.sessions[session_id]["memory"].clear()

    def generate_response(self, query: str) -> dict:
        """Execute the full RAG pipeline"""
        result = self.qa_chain({"question": query})
        return {
            "answer": result["answer"],
            "sources": result["sources"]
        }


In [39]:
qdrant_client = qdrant_store.get_client()
reranker = ReRanker().get_reranker()
hybrid_search = HybridSearch(
        qdrant_client=qdrant_client,
        embedding_model=embedding_model,
        sparse_embedding_model=sparse_model,
        reranker=reranker
)
   
load_dotenv()

llm = GeminiLLM(os.environ["GEMINI_API_KEY"], model_name="gemini-2.0-flash").get_llm()
    
    # Create pipeline
rag_pipeline = RAGPipeline(hybrid_search, llm)

c:\Users\Asus\anaconda3\envs\agent\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [67]:
# response = rag_pipeline.generate_response(qa_df["Câu hỏi"][0])
print("Answer:", response["answer"])
# print("Sources:", response["sources"])

Answer: **USER:** Trong trường hợp lắp đặt điện thoại cửa ở khu vực có giông bão thường xuyên, cần tuân thủ những biện pháp an toàn nào để tránh hư hỏng thiết bị?
**BOT (THINK):** Câu hỏi này liên quan đến các biện pháp an toàn khi lắp đặt điện thoại cửa ở khu vực có giông bão. Tôi cần tìm thông tin về các biện pháp phòng ngừa hoặc hướng dẫn lắp đặt trong điều kiện thời tiết khắc nghiệt.
**BOT (ACTION):**
```
ACTION: SEARCH("lắp đặt điện thoại cửa giông bão")
```


In [57]:
import string
from typing import List, Dict

def calculate_f1(rag_pipeline, qa_pairs: List[Dict[str, str]]) -> Dict[str, float]:
    """
    Tính toán Precision, Recall, F1-score cho hệ thống RAG dựa trên bộ dữ liệu QA.
    
    Args:
        rag_pipeline: Đối tượng RAGPipeline đã được khởi tạo
        qa_pairs: Danh sách các cặp câu hỏi-câu trả lời mẫu
    
    Returns:
        Dict chứa các metric đánh giá
    """
    tp = 0  # True Positive
    fp = 0  # False Positive
    fn = 0  # False Negative
    refusal_answer = "Tôi không thể hỗ trợ cho bạn về câu hỏi này"
    
    for pair in qa_pairs:
        question = pair["Câu hỏi"]
        expected_answer = pair["Câu trả lời"]
        is_product_question = (expected_answer != refusal_answer)
        
        # Lấy câu trả lời từ chatbot
        try:
            response = rag_pipeline.generate_response(question)
            generated_answer = response["answer"].strip()
            print(generated_answer)
        except Exception as e:
            print(f"Lỗi khi xử lý câu hỏi: {question}")
            generated_answer = ""
        
        # Chuẩn hóa văn bản để so sánh
        def normalize_text(text: str) -> str:
            text = text.lower().translate(str.maketrans("", "", string.punctuation))
            return " ".join(text.split())
        
        # Logic đánh giá
        if is_product_question:
            expected_norm = normalize_text(expected_answer)
            generated_norm = normalize_text(generated_answer)
            
            if expected_norm in generated_norm:
                tp += 1
            else:
                fn += 1  # Không trả lời được câu hỏi thuộc phạm vi
        else:
            expected_norm = normalize_text(refusal_answer)
            generated_norm = normalize_text(generated_answer)
            
            if expected_norm == generated_norm:
                tp += 1
            else:
                fp += 1  # Trả lời câu hỏi ngoài phạm vi
    
    # Tính toán các chỉ số
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        "precision": round(precision, 4),
        "recall": round(recall, 4),
        "f1": round(f1, 4),
        "total_samples": len(qa_pairs),
        "true_positives": tp,
        "false_positives": fp,
        "false_negatives": fn
    }

# Ví dụ sử dụng
    # Định nghĩa dữ liệu QA từ file Excel (hardcode cho ví dụ)
qa_pairs = [
        {
            "Câu hỏi": qa_df["Câu hỏi"][0],
            "Câu trả lời": qa_df["Câu trả lời"][0]
        }
        # ... Thêm các câu hỏi khác từ file Excel
]
    
    # Khởi tạo RAG pipeline (sử dụng code từ phần trước)
    # ... (code khởi tạo từ phần Usage Example)
    
    # Tính toán metrics
metrics = calculate_f1(rag_pipeline, qa_pairs)
    
print("\nKết quả đánh giá:")
print(f"- Precision: {metrics['precision']}")
print(f"- Recall: {metrics['recall']}")
print(f"- F1-score: {metrics['f1']}")
print(f"- Số mẫu: {metrics['total_samples']}")
print(f"- True Positives (TP): {metrics['true_positives']}")
print(f"- False Positives (FP): {metrics['false_positives']}")
print(f"- False Negatives (FN): {metrics['false_negatives']}")



> Entering new LLMChain chain...
Prompt after formatting:
"
Bạn là **ProductReportBot**, một trợ lý ảo chuyên về **hỏi–đáp tài liệu phân tích báo cáo sản phẩm**. Nhiệm vụ của bạn:

1. **Chỉ** trả lời các câu hỏi liên quan đến nội dung của tài liệu phân tích báo cáo sản phẩm đã được cung cấp.  
2. **Không** trả lời hoặc hỗ trợ bất kỳ câu hỏi nào ngoài phạm vi đó (ví dụ: hỏi về thời tiết, cuộc sống cá nhân, kiến thức chung…).  
3. Nếu được hỏi điều gì không liên quan, hãy trả lời lịch sự:  
   > “Xin lỗi, tôi chỉ được phép trả lời các câu hỏi về tài liệu phân tích báo cáo sản phẩm.”  
4. Duy trì một **bộ đếm hình phạt** nội bộ cho các câu trả lời sai:
   - Bắt đầu từ 0.  
   - Nếu người dùng cho rằng câu trả lời của bạn không đúng hoặc không được hỗ trợ bởi tài liệu, tăng bộ đếm lên 1.  
   - Sau mỗi lần tăng, bạn phải:
     1. Thừa nhận sai sót:  
        > “Tôi hiểu, tôi sẽ sửa lại.”  
     2. Chạy lại quy trình ReAct để đưa ra câu trả lời đúng.  
   - Nếu bộ đếm hình phạt đạt **3**,

In [51]:
print("\nKết quả đánh giá:")
print(f"- Precision: {metrics['precision']}")
print(f"- Recall: {metrics['recall']}")
print(f"- F1-score: {metrics['f1']}")
print(f"- Số mẫu: {metrics['total_samples']}")
print(f"- True Positives (TP): {metrics['true_positives']}")
print(f"- False Positives (FP): {metrics['false_positives']}")
print(f"- False Negatives (FN): {metrics['false_negatives']}")


Kết quả đánh giá:
- Precision: 0.5
- Recall: 1.0
- F1-score: 0.6667
- Số mẫu: 2
- True Positives (TP): 1
- False Positives (FP): 1
- False Negatives (FN): 0


In [47]:
scores = []
for id, row in qa_df.iterrows():
    response = rag_pipeline.generate_response(row["Câu hỏi"])
    answer = response["answer"]
    score = calculate_f1(row["Câu trả lời"], answer)
    scores.append(score)
qa_df["Điểm"] = score



> Entering new LLMChain chain...
Prompt after formatting:
"
Bạn là **ProductReportBot**, một trợ lý ảo chuyên về **hỏi–đáp tài liệu phân tích báo cáo sản phẩm**. Nhiệm vụ của bạn:

1. **Chỉ** trả lời các câu hỏi liên quan đến nội dung của tài liệu phân tích báo cáo sản phẩm đã được cung cấp.  
2. **Không** trả lời hoặc hỗ trợ bất kỳ câu hỏi nào ngoài phạm vi đó (ví dụ: hỏi về thời tiết, cuộc sống cá nhân, kiến thức chung…).  
3. Nếu được hỏi điều gì không liên quan, hãy trả lời lịch sự:  
   > “Xin lỗi, tôi chỉ được phép trả lời các câu hỏi về tài liệu phân tích báo cáo sản phẩm.”  
4. Duy trì một **bộ đếm hình phạt** nội bộ cho các câu trả lời sai:
   - Bắt đầu từ 0.  
   - Nếu người dùng cho rằng câu trả lời của bạn không đúng hoặc không được hỗ trợ bởi tài liệu, tăng bộ đếm lên 1.  
   - Sau mỗi lần tăng, bạn phải:
     1. Thừa nhận sai sót:  
        > “Tôi hiểu, tôi sẽ sửa lại.”  
     2. Chạy lại quy trình ReAct để đưa ra câu trả lời đúng.  
   - Nếu bộ đếm hình phạt đạt **3**,

In [48]:
qa_df

,Câu hỏi,Câu trả lời,Điểm
0,Điện thoại cửa có khả năng chống bụi/nước đạt ...,IP54.,NaN
1,Model điện thoại cửa nào được đề cập trong tài...,VL-V555.,NaN
2,Điều khiển chính cần được lắp cách xa điện tho...,5 mét.,NaN
3,Phụ kiện nào được sử dụng để cố định dây nguồn?,Đai giữ dây.,NaN
4,Nguồn điện yêu cầu cho bộ cấp điện VL-PS240 là...,220-240V AC.,NaN
...,...,...,...
83,Cây tre có nguồn gốc từ đâu?,Tôi không thể hỗ trợ cho bạn về câu hỏi này,NaN
84,Hệ điều hành Android thuộc sở hữu công ty nào?,Tôi không thể hỗ trợ cho bạn về câu hỏi này,NaN
85,Chim cánh cụt sống chủ yếu ở đâu?,Tôi không thể hỗ trợ cho bạn về câu hỏi này,NaN
86,Màu sắc cơ bản gồm những màu gì?,Tôi không thể hỗ trợ cho bạn về câu hỏi này,NaN
